# Импорт

In [9]:
import numpy as np
import pandas as pd
import spacy
nlp = spacy.load("ru_core_news_sm")
import tensorflow as tf
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc, mean_squared_error
from sklearn.model_selection import train_test_split
import tensorflow as tf
from json import load


# Формирование выборки

In [10]:
not_discriptors = list()
with open('../../Data/Words/word_pairs_not_disc.txt', 'r', encoding='utf-8') as txt:
    for i in txt:
        not_discriptors.append(i.replace('\n', ''))

with open('../../Data/Events/type_of_events.json', 'r', encoding='utf-8') as js:
    events = load(js)

with open('../../Data/Datasets/data.json', 'r', encoding='utf-8') as js:
    discriptors = load(js)


In [11]:
dict_of_disc = {'expession': [], 'is_discr': []}
for subject in discriptors.values():
    for d in sum(subject.values(), []):
        dict_of_disc['expession'].append(nlp(d).vector)
        dict_of_disc['is_discr'].append(True)

for subject in not_discriptors:
    dict_of_disc['expession'].append(nlp(subject).vector)
    dict_of_disc['is_discr'].append(False)



In [12]:
features = np.array(dict_of_disc['expession'])
labels = np.array(list(map(int, dict_of_disc['is_discr'])))


In [13]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=71)


# Создание модели 1

In [48]:
model_class1 = tf.keras.models.Sequential()
model_class1.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*1.5), activation='relu',input_shape=(X_train[0].shape[0],), name='input'))
model_class1.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*2), activation='relu', name='hidden_layer_2'))
model_class1.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*3), activation='relu', name='hidden_layer_3'))
model_class1.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*2), activation='sigmoid', name='hidden_layer_4', use_bias=True))
model_class1.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*1.5), activation='sigmoid', name='hidden_layer_5', use_bias=True))
model_class1.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*0.5), activation='sigmoid', name='hidden_layer_6', use_bias=True))
model_class1.add(tf.keras.layers.Dense(1, name='output', activation='sigmoid'))
model_class1.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Dense)               (None, 144)               13968     
                                                                 
 hidden_layer_2 (Dense)      (None, 192)               27840     
                                                                 
 hidden_layer_3 (Dense)      (None, 288)               55584     
                                                                 
 hidden_layer_4 (Dense)      (None, 192)               55488     
                                                                 
 hidden_layer_5 (Dense)      (None, 144)               27792     
                                                                 
 hidden_layer_6 (Dense)      (None, 48)                6960      
                                                                 
 output (Dense)              (None, 1)                

# Обучение MSE

In [15]:
model_class1.compile(
    loss='mse',
    optimizer='Adam',
    metrics=[
        'mse',
    ]
)


In [16]:
model_class1.fit(
    X_train, 
    y_train, 
    validation_split=0.2, 
    epochs=20,
    batch_size=2
)


Epoch 1/20
319/319 [==============================] - 2s 3ms/step - loss: 0.1315 - mse: 0.1315 - val_loss: 0.0812 - val_mse: 0.0812
Epoch 2/20
319/319 [==============================] - 1s 2ms/step - loss: 0.0709 - mse: 0.0709 - val_loss: 0.1100 - val_mse: 0.1100
Epoch 3/20
319/319 [==============================] - 1s 2ms/step - loss: 0.0625 - mse: 0.0625 - val_loss: 0.0682 - val_mse: 0.0682
Epoch 4/20
319/319 [==============================] - 1s 2ms/step - loss: 0.0543 - mse: 0.0543 - val_loss: 0.0562 - val_mse: 0.0562
Epoch 5/20
319/319 [==============================] - 1s 2ms/step - loss: 0.0599 - mse: 0.0599 - val_loss: 0.0993 - val_mse: 0.0993
Epoch 6/20
319/319 [==============================] - 1s 2ms/step - loss: 0.0714 - mse: 0.0714 - val_loss: 0.0894 - val_mse: 0.0894
Epoch 7/20
319/319 [==============================] - 1s 2ms/step - loss: 0.0605 - mse: 0.0605 - val_loss: 0.0859 - val_mse: 0.0859
Epoch 8/20
319/319 [==============================] - 1s 2ms/step - loss: 0.

In [17]:
predict = model_class1.predict(X_test)


9/9 [==============================] - 0s 1ms/step


In [18]:
mean_squared_error(predict, y_test)


0.057596620252465286

In [19]:
predictA = model_class1.predict(features)
print('Accuracy')
print(accuracy_score(labels, predictA > 0.5))
print('Confusion matrix')
print(confusion_matrix(labels, predictA > 0.5))
print('Precision, Recall, F\n', classification_report(
    labels, predictA > 0.5))


34/34 [==============================] - 0s 1ms/step
Accuracy
0.9577464788732394
Confusion matrix
[[320  38]
 [  7 700]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.98      0.89      0.93       358
           1       0.95      0.99      0.97       707

    accuracy                           0.96      1065
   macro avg       0.96      0.94      0.95      1065
weighted avg       0.96      0.96      0.96      1065



In [20]:
print('Accuracy')
print(accuracy_score(y_test, predict > 0.98))
print('Confusion matrix')
print(confusion_matrix(y_test, predict > 0.98))
print('Precision, Recall, F\n', classification_report(
    y_test, predict > 0.98))


Accuracy
0.947565543071161
Confusion matrix
[[ 85  11]
 [  3 168]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.97      0.89      0.92        96
           1       0.94      0.98      0.96       171

    accuracy                           0.95       267
   macro avg       0.95      0.93      0.94       267
weighted avg       0.95      0.95      0.95       267



In [21]:
tn, fp, fn, tp = confusion_matrix(y_test, predict > 0.9).ravel()
(tn, fp, fn, tp)


(84, 12, 3, 168)

In [25]:
model_class1.predict(np.array([nlp('рекомендации, чтобы поддерживать внутреннюю').vector])), model_class1.predict(np.array([nlp('чтобы поддерживать внутреннюю').vector])), model_class1.predict(np.array([nlp('поддерживать внутреннюю структуру').vector]))



1/1 [==============================] - 0s 18ms/step


(array([[0.9891076]], dtype=float32),
 array([[0.9891076]], dtype=float32),
 array([[0.9891076]], dtype=float32))

# Обучение на Binary Crossentropy

In [34]:
model_class1.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer='Adam',
    metrics=[
        'mse',
    ]
)


In [35]:
model_class1.fit(
    X_train, 
    y_train, 
    validation_split=0.2, 
    epochs=20,
    batch_size=2
)


Epoch 1/20
319/319 [==============================] - 2s 3ms/step - loss: 0.4278 - mse: 0.1344 - val_loss: 0.2895 - val_mse: 0.0800
Epoch 2/20
319/319 [==============================] - 1s 2ms/step - loss: 0.2407 - mse: 0.0654 - val_loss: 0.2314 - val_mse: 0.0639
Epoch 3/20
319/319 [==============================] - 1s 2ms/step - loss: 0.1735 - mse: 0.0445 - val_loss: 0.2047 - val_mse: 0.0570
Epoch 4/20
319/319 [==============================] - 1s 2ms/step - loss: 0.1303 - mse: 0.0302 - val_loss: 0.2428 - val_mse: 0.0673
Epoch 5/20
319/319 [==============================] - 1s 2ms/step - loss: 0.1246 - mse: 0.0268 - val_loss: 0.2023 - val_mse: 0.0442
Epoch 6/20
319/319 [==============================] - 1s 2ms/step - loss: 0.1277 - mse: 0.0276 - val_loss: 0.1529 - val_mse: 0.0349
Epoch 7/20
319/319 [==============================] - 1s 2ms/step - loss: 0.0912 - mse: 0.0182 - val_loss: 0.1672 - val_mse: 0.0372
Epoch 8/20
319/319 [==============================] - 1s 2ms/step - loss: 0.

In [36]:
predict = model_class1.predict(X_test)


9/9 [==============================] - 0s 2ms/step


In [37]:
mean_squared_error(predict, y_test)


0.044182525926360405

In [38]:
predictA = model_class1.predict(features)
print('Accuracy')
print(accuracy_score(labels, predictA > 0.5))
print('Confusion matrix')
print(confusion_matrix(labels, predictA > 0.5))
print('Precision, Recall, F\n', classification_report(
    labels, predictA > 0.5))


34/34 [==============================] - 0s 1ms/step
Accuracy
0.9755868544600939
Confusion matrix
[[339  19]
 [  7 700]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.98      0.95      0.96       358
           1       0.97      0.99      0.98       707

    accuracy                           0.98      1065
   macro avg       0.98      0.97      0.97      1065
weighted avg       0.98      0.98      0.98      1065



In [39]:
print('Accuracy')
print(accuracy_score(y_test, predict > 0.98))
print('Confusion matrix')
print(confusion_matrix(y_test, predict > 0.98))
print('Precision, Recall, F\n', classification_report(
    y_test, predict > 0.98))


Accuracy
0.9550561797752809
Confusion matrix
[[ 88   8]
 [  4 167]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.96      0.92      0.94        96
           1       0.95      0.98      0.97       171

    accuracy                           0.96       267
   macro avg       0.96      0.95      0.95       267
weighted avg       0.96      0.96      0.95       267



In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, predict > 0.9).ravel()
(tn, fp, fn, tp)


(84, 12, 3, 168)

In [ ]:
model_class1.predict(np.array([nlp('рекомендации, чтобы поддерживать внутреннюю').vector])), model_class1.predict(np.array([nlp('чтобы поддерживать внутреннюю').vector])), model_class1.predict(np.array([nlp('поддерживать внутреннюю структуру').vector]))



1/1 [==============================] - 0s 18ms/step


(array([[0.9891076]], dtype=float32),
 array([[0.9891076]], dtype=float32),
 array([[0.9891076]], dtype=float32))

# Обучение на Poison

In [40]:
model_class1.compile(
    loss=tf.keras.losses.poisson,
    optimizer='Adam',
    metrics=[
        'mse',
    ]
)


In [41]:
model_class1.fit(
    X_train, 
    y_train, 
    validation_split=0.2, 
    epochs=20,
    batch_size=2
)


Epoch 1/20
319/319 [==============================] - 2s 3ms/step - loss: 0.6954 - mse: 0.0111 - val_loss: 0.7846 - val_mse: 0.0504
Epoch 2/20
319/319 [==============================] - 1s 2ms/step - loss: 0.6922 - mse: 0.0078 - val_loss: 0.8116 - val_mse: 0.0556
Epoch 3/20
319/319 [==============================] - 1s 2ms/step - loss: 0.6905 - mse: 0.0062 - val_loss: 0.8132 - val_mse: 0.0556
Epoch 4/20
319/319 [==============================] - 1s 2ms/step - loss: 0.6904 - mse: 0.0062 - val_loss: 0.8174 - val_mse: 0.0557
Epoch 5/20
319/319 [==============================] - 1s 2ms/step - loss: 0.6905 - mse: 0.0062 - val_loss: 0.8150 - val_mse: 0.0557
Epoch 6/20
319/319 [==============================] - 1s 2ms/step - loss: 0.6905 - mse: 0.0062 - val_loss: 0.8143 - val_mse: 0.0557
Epoch 7/20
319/319 [==============================] - 1s 2ms/step - loss: 0.6905 - mse: 0.0062 - val_loss: 0.8141 - val_mse: 0.0557
Epoch 8/20
319/319 [==============================] - 1s 2ms/step - loss: 0.

In [42]:
predict = model_class1.predict(X_test)


9/9 [==============================] - 0s 2ms/step


In [43]:
mean_squared_error(predict, y_test)


0.0524771339190995

In [44]:
predictA = model_class1.predict(features)
print('Accuracy')
print(accuracy_score(labels, predictA > 0.5))
print('Confusion matrix')
print(confusion_matrix(labels, predictA > 0.5))
print('Precision, Recall, F\n', classification_report(
    labels, predictA > 0.5))


34/34 [==============================] - 0s 2ms/step
Accuracy
0.9746478873239437
Confusion matrix
[[344  14]
 [ 13 694]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.96      0.96      0.96       358
           1       0.98      0.98      0.98       707

    accuracy                           0.97      1065
   macro avg       0.97      0.97      0.97      1065
weighted avg       0.97      0.97      0.97      1065



In [45]:
print('Accuracy')
print(accuracy_score(y_test, predict > 0.98))
print('Confusion matrix')
print(confusion_matrix(y_test, predict > 0.98))
print('Precision, Recall, F\n', classification_report(
    y_test, predict > 0.98))


Accuracy
0.951310861423221
Confusion matrix
[[ 90   6]
 [  7 164]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.93      0.94      0.93        96
           1       0.96      0.96      0.96       171

    accuracy                           0.95       267
   macro avg       0.95      0.95      0.95       267
weighted avg       0.95      0.95      0.95       267



In [46]:
tn, fp, fn, tp = confusion_matrix(y_test, predict > 0.9).ravel()
(tn, fp, fn, tp)


(89, 7, 7, 164)

In [47]:
model_class1.predict(np.array([nlp('рекомендации, чтобы поддерживать внутреннюю').vector])), model_class1.predict(np.array([nlp('чтобы поддерживать внутреннюю').vector])), model_class1.predict(np.array([nlp('поддерживать внутреннюю структуру').vector]))



1/1 [==============================] - 0s 18ms/step


(array([[0.99341315]], dtype=float32),
 array([[0.9934132]], dtype=float32),
 array([[0.99341327]], dtype=float32))